# TWITTER SENTIMENT ANALYSIS ON COVID DATA - WEBSCRAPING

## Importing Packages and Connecting Data

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
import snscrape.modules.twitter as sntwitter
from tqdm.notebook import tqdm
from pathlib import Path  
import re
import string
from transformers import AutoTokenizer
#add package
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [30]:
#package for social network scraping
#pip install snscrape

## Web Scraping for CovidVaccine  related tweets 

In [31]:
query = "python"

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    print(vars(tweet))
    break

{'url': 'https://twitter.com/BlackWomenInAI/status/1589943934788440064', 'date': datetime.datetime(2022, 11, 8, 11, 32, 28, tzinfo=datetime.timezone.utc), 'content': "What is A.I.? That's A.I.  #100DaysofCode Rewind ⏪⏪  #music  #BlackTechTwitter #BWIAI #Apple #WomenWhoCode  #REWIND  #Tech #STEM #Python\n@BlackWomenInAI @nas #HipHop https://t.co/JanFVBTzkZ", 'renderedContent': "What is A.I.? That's A.I.  #100DaysofCode Rewind ⏪⏪  #music  #BlackTechTwitter #BWIAI #Apple #WomenWhoCode  #REWIND  #Tech #STEM #Python\n@BlackWomenInAI @nas #HipHop https://t.co/JanFVBTzkZ", 'id': 1589943934788440064, 'user': User(username='BlackWomenInAI', id=944826755793281024, displayname='Black Women in A.I.', description='Amplifying the voices of Black Women In Artificial Intelligence. We Are Here- Educate-Engage-Embrace-Empower - Join us today .  - Global', rawDescription='Amplifying the voices of Black Women In Artificial Intelligence. We Are Here- Educate-Engage-Embrace-Empower - Join us today .  - Glob

In [32]:

# webscaping from 2020 to 2021
tweets = []
limit = 1000

query = "(covid+vaccine OR Nuvaxovid OR Novavax OR pfizer OR pfizer-biontech OR pfizerbiontech OR moderna OR astrazeneca OR astra+zeneca) -availability -free -slot -slots -trade -trades (#covidvaccine OR #CovidVaccine OR #vaccine OR #covid OR #Covid19Vaccine OR #CoronaVirusVaccine ) lang:en until:2021-12-8 since:2020-12-08 -filter:links -filter:replies"

for tweet in tqdm(sntwitter.TwitterSearchScraper(query).get_items(),total = limit):
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date,tweet.user.username,tweet.content,tweet.url,tweet.user.location])
vaccine_df_2020_2021 = pd.DataFrame(tweets,columns=['Date','User','Tweet','Tweet_URL','Location'])
vaccine_df_2020_2021.head()


  0%|          | 0/1000 [00:00<?, ?it/s]

,Date,User,Tweet,Tweet_URL,Location
0,2021-12-07 23:58:57+00:00,TPedherney,In line to get my booster #vaccine #Booster #P...,https://twitter.com/TPedherney/status/14683694...,"Burlington, ON"
1,2021-12-07 23:32:39+00:00,cwillits30,Officially boosted! First done of Moderna afte...,https://twitter.com/cwillits30/status/14683628...,"Phoenix, AZ"
2,2021-12-07 23:29:36+00:00,MA_CEAL,#COVIDMythBustingTuesdays! \n\nMyth: Doesn’t t...,https://twitter.com/MA_CEAL/status/14683620841...,"Boston, Massachusetts"
3,2021-12-07 23:08:19+00:00,WebAeneas,#vaccinated another great idea you can sue any...,https://twitter.com/WebAeneas/status/146835672...,
4,2021-12-07 23:02:41+00:00,TheAmirImani,"I don't know if this is a ""3 dose #vaccine"" or...",https://twitter.com/TheAmirImani/status/146835...,Toronto


In [33]:
vaccine_df_2020_2021_newline_cleaned = vaccine_df_2020_2021.replace(r'\n',' ', regex=True).replace(r'\r',' ', regex=True).replace(r'\t',' ', regex=True)
vaccine_df_2020_2021_newline_cleaned1 = vaccine_df_2020_2021_newline_cleaned.replace(r'\\r',' ', regex=True).replace(r'\\n',' ', regex=True).replace(r'\\t',' ',regex=True)
vaccine_df_2020_2021_white_space = vaccine_df_2020_2021_newline_cleaned1.replace(r' +',' ',regex=True)
vaccine_df_2020_2021_white_space.head()

,Date,User,Tweet,Tweet_URL,Location
0,2021-12-07 23:58:57+00:00,TPedherney,In line to get my booster #vaccine #Booster #P...,https://twitter.com/TPedherney/status/14683694...,"Burlington, ON"
1,2021-12-07 23:32:39+00:00,cwillits30,Officially boosted! First done of Moderna afte...,https://twitter.com/cwillits30/status/14683628...,"Phoenix, AZ"
2,2021-12-07 23:29:36+00:00,MA_CEAL,#COVIDMythBustingTuesdays! Myth: Doesn’t the #...,https://twitter.com/MA_CEAL/status/14683620841...,"Boston, Massachusetts"
3,2021-12-07 23:08:19+00:00,WebAeneas,#vaccinated another great idea you can sue any...,https://twitter.com/WebAeneas/status/146835672...,
4,2021-12-07 23:02:41+00:00,TheAmirImani,"I don't know if this is a ""3 dose #vaccine"" or...",https://twitter.com/TheAmirImani/status/146835...,Toronto


In [34]:
vaccine_df_2020_2021['Tweet'][6]

'About 1600 people die each day from #COVID in the US.\n\nMost are #unvaccinated \n\nWe have a medication. It’s called a #vaccine &amp; it works! And btw. It’s in surplus!\n\nIs politics, party affiliation &amp; misinformation really worth one’s life?'

In [35]:
vaccine_df_2020_2021_white_space['Tweet'][6]

'About 1600 people die each day from #COVID in the US. Most are #unvaccinated We have a medication. It’s called a #vaccine &amp; it works! And btw. It’s in surplus! Is politics, party affiliation &amp; misinformation really worth one’s life?'

In [36]:
vaccine_df_2020_2021_white_space.duplicated(subset=['Tweet']).sum()

4

In [37]:
vaccine_df_clean=vaccine_df_2020_2021_white_space.drop_duplicates(subset=['Tweet'])
vaccine_df_clean.shape

(996, 5)

In [38]:
filepath = Path('data/vaccine_output_2020_2021.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
vaccine_df_clean.to_csv(filepath)

In [39]:
#Webscraping from 2021 to 2022
tweets = []
query = "(covid+vaccine OR Nuvaxovid OR Novavax OR pfizer OR pfizer-biontech OR pfizerbiontech OR moderna OR astrazeneca OR astra+zeneca) -availability -free -slot -slots -trade -trades (#covidvaccine OR #CovidVaccine OR #vaccine OR #covid OR #Covid19Vaccine OR #CoronaVirusVaccine ) lang:en until:2022-12-8 since:2021-12-08 -filter:links -filter:replies"

for tweet in tqdm(sntwitter.TwitterSearchScraper(query).get_items(),total = limit):
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date,tweet.user.username,tweet.content,tweet.url,tweet.user.location])
vaccine_df_2021_2022 = pd.DataFrame(tweets,columns=['Date','User','Tweet','Tweet_URL','Location'])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [40]:
vaccine_df_2021_2022_newline_cleaned = vaccine_df_2021_2022.replace(r'\n',' ', regex=True).replace(r'\r',' ', regex=True).replace(r'\t',' ', regex=True)
vaccine_df_2021_2022_newline_cleaned1 = vaccine_df_2021_2022_newline_cleaned.replace(r'\\r',' ', regex=True).replace(r'\\n',' ', regex=True).replace(r'\\t',' ',regex=True)
vaccine_df_2021_2022_white_space = vaccine_df_2021_2022_newline_cleaned1.replace(r' +',' ',regex=True)
vaccine_df_2021_2022_white_space.duplicated(subset=['Tweet']).sum()

2

In [41]:
vaccine_df_2021_2022_clean=vaccine_df_2021_2022_white_space.drop_duplicates(subset=['Tweet'])
vaccine_df_2021_2022_clean.shape

(998, 5)

In [42]:
filepath = Path('data/vaccine_output_2021_2022.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
vaccine_df_2021_2022_clean.to_csv(filepath)